# RAG Operations in Pixeltable

Authors: Marcel Kornacker, Aaron Siegel

In this tutorial, we'll explore Pixeltable's flexible handling of RAG operations on unstructured text. In a traditional AI workflow, such operations might be implemented as a Python script that runs on a periodic schedule or in response to certain events. In Pixeltable, as with everything else, they are implemented as persistent table operations that update incrementally as new data becomes available. In our tutorial workflow, we'll chunk Wikipedia articles in various ways with a document splitter, then apply several kinds of embeddings to the chunks.

This tutorial is an interactive notebook, and you can follow along by executing the cells one at a time. You'll need to install Pixeltable first, if you haven't already (TODO: link to installation tutorial?), along with the Huggingface `sentence_transformers` package.

## Set Up the Table Structure

We start by creating a Pixeltable client object.

In [ ]:
import pixeltable as pxt
cl = pxt.Client()

The next few commands create a Pixeltable workspace `rag_demo` (if it doesn't already exist) and set up the table structure for our new workflow.

In [ ]:
# Create the Pixeltable workspace
cl.create_dir('rag_demo', ignore_errors=True)

# Clean the database to ensure we're using fresh table instances
# (in case this demo has been run before)
cl.drop_table('rag_demo.short_char_chunks', ignore_errors=True)
cl.drop_table('rag_demo.short_chunks', ignore_errors=True)
cl.drop_table('rag_demo.chunks', ignore_errors=True)
cl.drop_table('rag_demo.sentences', ignore_errors=True)
cl.drop_table('rag_demo.docs', ignore_errors=True)

## Creating Tables and Views

Now we'll create the tables that represent our workflow, starting with a table to hold references to source documents. The table contains a single column `source_doc` whose elements have type `pxt.DocumentType`, representing a general document instance. In this tutorial, we'll be working with HTML documents, but Pixeltable supports a range of other document types, such as Markdown and PDF.

In [ ]:
docs = cl.create_table('rag_demo.docs', {'source_doc': pxt.DocumentType()})

If we take a peek at the `docs` table, we see its very simple structure. Run the following command:

In [ ]:
docs

Next we create a view to represent chunks of our HTML documents. A Pixeltable view is a virtual table, which is dynamically derived from a source table by applying a transformation and/or selecting a subset of data. In this case, our view represents a one-to-many transformation from source documents into individual sentences. This is achived using Pixeltable's built-in `DocumentSplitter` class.

Note that the `docs` table is currently empty, so creating this view doesn't actually *do* anything yet: it simply defines an operation that we want Pixeltable to execute when it sees new data.

In [ ]:
from pixeltable.iterators.document import DocumentSplitter

sentences = cl.create_view(
    'rag_demo.sentences',  # Name of the view
    docs,  # Table from which the view is derived
    iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'sentence',  # Chunk docs into sentences
        'metadata': 'title,headings,sourceline',
    })

Let's take a peek at the new `sentences` view. Run the following command:

In [ ]:
sentences

We see that `sentences` inherits the `source_doc` column from `docs`, together with some new fields:
- `pos`: The position in the source document where the sentence appears.
-  `text`: The text of the sentence.
- `title`, `headings`, and `sourceline`: The metadata we requested when we set up the view.

## Data Ingestion

Ok, now it's time to insert some data into our workflow. A document in Pixeltable is just a URL; the following command inserts a single row into the `docs` table with the `source_doc` field set to the specified URL:

In [ ]:
docs.insert(source_doc='https://en.wikipedia.org/wiki/Marc_Chagall')

We can see that two things happened. First, a single row was inserted into `docs`, containing the URL representing our source document. Then, the view `sentences` was incrementally updated by applying the `DocumentSplitter` according to the definition of the view. This illustrates an important principle in Pixeltable: anytime Pixeltable sees new data, the update is incrementally propagated to any downstream views or computed columns.

We can see the effect of the insertion with the `select` command. There's a single row in `docs`:

In [ ]:
docs.select(docs.source_doc.fileurl).show()

And here are the first 50 rows in `sentences`. The content of the article is broken into individual sentences, as expected.

In [ ]:
sentences.select(sentences.text, sentences.headings).show(50)

## Experimenting with Chunking

Of course, chunking into sentences isn't the only way to split a document. Perhaps we want to experiment with different chunking methodologies, in order to see which one performs best in a particular application. Pixeltable makes it easy to do this, by creating several views of the same source table. Here are a few examples. Notice that as each new view is created, it is initially populated from the data already in `docs`.

In [ ]:
chunks = cl.create_view(
    'rag_demo.chunks', docs, iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'paragraph,token_limit',
        'limit': 2048,
        'overlap': 0,
        'metadata': 'title,headings,sourceline',
    })

In [ ]:
short_chunks = cl.create_view(
    'rag_demo.short_chunks', docs, iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'paragraph,token_limit',
        'limit': 72,
        'overlap': 0,
        'metadata': 'title,headings,sourceline',
    })

In [ ]:
short_char_chunks = cl.create_view(
    'rag_demo.short_char_chunks', docs, iterator_class=DocumentSplitter,
    iterator_args={
        'document': docs.source_doc,
        'separators': 'paragraph,char_limit',
        'limit': 72,
        'overlap': 0,
        'metadata': 'title,headings,sourceline',
    })

In [ ]:
chunks.select(chunks.text, chunks.headings).show(50)

In [ ]:
short_chunks.select(short_chunks.text, short_chunks.headings).show(50)

In [ ]:
short_char_chunks.select(short_char_chunks.text, short_char_chunks.headings).show(50)

Now let's add a few more documents to our workflow. Notice how all of the downstream views are updated incrementally, processing just the new documents as they are inserted.

In [ ]:
urls = [
    'https://en.wikipedia.org/wiki/Pierre-Auguste_Renoir',
    'https://en.wikipedia.org/wiki/Henri_Matisse',
    'https://en.wikipedia.org/wiki/Marcel_Duchamp'
]
docs.insert([{'source_doc': url} for url in urls])

## Further Experiments

This is a good time to mention another important guiding principle of Pixeltable. The preceding examples all used the built-in `DocumentSplitter` class with various configurations. That's probably fine as a first cut or to prototype an application quickly, and it might be sufficient for some applications. But other applications might want to do more sophisticated kinds of chunking, implementing their own specialized logic or leveraging third-party tools. Pixeltable imposes no constraints on the AI or RAG operations a workflow uses: the iterator interface is highly general, and it's easy to implement new operations or adapt existing code or third-party tools into the Pixeltable workflow. Pixeltable manages *data* storage and workflow that is common to all such operations: above the data plane, it's fully general and extensible.

TODO: Example of a custom splitter?

## Adding embeddings

Next, let's look at how embeddings can be added seamlessly to existing Pixeltable workflows. To compute our embeddings, we'll use the Huggingface `sentence_transformer` package, running it over the `chunks` view that broke our documents up into larger paragraphs. Pixeltable has a built-in `sentence_transformer` adapter, and all we have to do is add a new column that leverages it. Pixeltable takes care of the rest, applying the new column to all existing data in the view.

You may see a lot of output, especially if you're using `sentence_transformers` for the first time.

In [ ]:
from pixeltable.functions.huggingface import sentence_transformer, clip

minilm_model_id = 'paraphrase-MiniLM-L6-v2'

chunks.add_column(minilm_embedding=sentence_transformer(chunks.text, model_id=minilm_model_id))

The new column is a *computed column*: it is defined as a function on top of existing data and updated incrementally as new data are added to the workflow. Let's have a look at how the new column affected the `chunks` view.

In [ ]:
chunks

In [ ]:
chunks.head()

Similarly, we might want to add a CLIP embedding to our workflow; once again, it's just another computed column:

In [ ]:
clip_model_id = 'openai/clip-vit-base-patch32'

chunks.add_column(clip_embedding=clip(text=chunks.text, model_id=clip_model_id))

In [ ]:
chunks

In [ ]:
chunks.head()